<a href="https://colab.research.google.com/github/chaeeewon/hello-world/blob/master/working_mobilenet_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras import datasets, layers, models


In [0]:
# data preprocessing
AUTOTUNE = tf.data.experimental.AUTOTUNE

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

total_train_step = len(train_images)
training_batch_size = 10

# input image 32*32를 224*224로 resize 실행 --> VGG-16의 input size를 맞추기 위해
# 상세한 이유는 하기의 레이어 구조를 확인
def resize_image(train_image, train_label):
  train_image = tf.image.resize(train_image, (224,224))
  return train_image, train_label

# dataset을 image 순서로 나누어 저장하는 함수
dataset_cifar10 =  tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_cifar10 =  tf.data.Dataset.from_tensor_slices((test_images, test_labels))
# map 함수와 callback 함수를 이용하여 image resize
dataset_cifar10 = dataset_cifar10.map(resize_image, num_parallel_calls=AUTOTUNE)

# buffer_size(training image를 shuffle 하기 위한 buffer size)
# batch size(한 step에 넣을 input image 개수)
dataset_cifar10 = dataset_cifar10.shuffle(buffer_size=1000)
dataset_cifar10 = dataset_cifar10.batch(batch_size=training_batch_size, drop_remainder=True)
dataset_cifar10 = dataset_cifar10.prefetch(buffer_size=1000)

# test용 dataset 설정
test_cifar10 = test_cifar10.map(resize_image, num_parallel_calls=AUTOTUNE)
test_cifar10 = test_cifar10.batch(batch_size=training_batch_size, drop_remainder=True)


In [0]:
# [손혜정 / Hyejung Son] - 13:43                
def cnn_mobile_net(classes=10):
  input_shape = (224,224,3)
  img_input = layers.Input(shape=input_shape)
  x = img_input


  #depthwise conv + 1x1conv 같이
  x = layers.Conv2D(32,(3,3), padding='same',name='conv1_s2',strides=(2, 2))(x)
  x = layers.BatchNormalization(name='bn_conv1')(x)
  x = layers.Activation('relu')(x)


  x = layers.SeparableConv2D(32,(3,3), padding='same', name='conv2_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv2')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(64,(3,3), padding='same', name='conv3_dw_s2',strides=(2, 2))(x)
  x = layers.BatchNormalization(name='bn_conv3')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(128,(3,3), padding='same', name='conv4_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv4')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(128,(3,3), padding='same', name='conv5_dw_s2',strides=(2, 2))(x)
  x = layers.BatchNormalization(name='bn_conv5')(x)
  x = layers.Activation('relu')(x)


  x = layers.SeparableConv2D(256,(3,3), padding='same', name='conv6_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv6')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(256,(3,3), padding='same', name='conv7_dw_s2',strides=(2, 2))(x)
  x = layers.BatchNormalization(name='bn_conv7')(x)
  x = layers.Activation('relu')(x)


  x = layers.SeparableConv2D(512,(3,3), padding='same', name='conv8_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv8')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(512,(3,3), padding='same', name='conv9_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv9')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(512,(3,3), padding='same', name='conv10_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv10')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(512,(3,3), padding='same', name='conv11_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv11')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(512,(3,3), padding='same', name='conv12_dw_s1',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv12')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(512,(3,3), padding='same', name='conv13_dw_s2',strides=(2, 2))(x)
  x = layers.BatchNormalization(name='bn_conv13')(x)
  x = layers.Activation('relu')(x)

  x = layers.SeparableConv2D(1024,(3,3), padding='same', name='conv14_dw_s2',strides=(1, 1))(x)
  x = layers.BatchNormalization(name='bn_conv14')(x)
  x = layers.Activation('relu')(x)

  x = layers.AveragePooling2D((7,7))(x)
  x = layers.Flatten()(x)
  x = layers.Dense(1000, activation='relu')(x)
  x = layers.Dense(10, activation='softmax')(x)


  inputs = img_input
  model = models.Model(inputs, x, name='cnn_mobile_net')
  return model



In [0]:
# network class
def mobilenet(classes=10):
  input_shape = (224,224,3)
  img_input = tf.keras.Input(shape=input_shape)
  x = img_input
  x = layers.Conv2D(32, (3,3), (2,2), padding='same', name='conv1_1')(x)
  x = layers.BatchNormalization(name='bn_conv1')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(32, (3,3), (1, 1), padding='same', name='conv1_2')(x)
  x = layers.BatchNormalization(name='bn_conv2')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(64, (3,3), (2,2), padding='same', name='conv1_3')(x)
  x = layers.BatchNormalization(name='bn_conv3')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(128, (3,3), (1,1), padding='same', name='conv1_4')(x)
  x = layers.BatchNormalization(name='bn_conv4')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(128, (3,3), (2,2), padding='same', name='conv1_5')(x)
  x = layers.BatchNormalization(name='bn_conv5')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(256, (3,3), (1,1), padding='same', name='conv1_6')(x)
  x = layers.BatchNormalization(name='bn_conv6')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(256, (3,3), (2,2), padding='same', name='conv1_7')(x)
  x = layers.BatchNormalization(name='bn_conv7')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(512, (3,3), (1,1), padding='same', name='conv1_8')(x)
  x = layers.BatchNormalization(name='bn_conv8')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(512, (3,3), (1,1), padding='same', name='conv1_9')(x)
  x = layers.BatchNormalization(name='bn_conv9')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(512, (3,3), (1,1), padding='same', name='conv1_10')(x)
  x = layers.BatchNormalization(name='bn_conv10')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(512, (3,3), (1,1), padding='same', name='conv1_11')(x)
  x = layers.BatchNormalization(name='bn_conv11')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(512, (3,3), (2,2), padding='same', name='conv1_12')(x)
  x = layers.BatchNormalization(name='bn_conv12')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(512, (3,3), (2, 2), padding='same', name='conv1_13')(x)
  x = layers.BatchNormalization(name='bn_conv13')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.SeparableConv2D(1024, (3,3), (1,1), padding='same', name='conv1_14')(x)
  x = layers.BatchNormalization(name='bn_conv14')(x)
  x = layers.Activation('relu')(x)
  
  x = tf.compat.v1.layers.AveragePooling2D((7,7), strides=(1,1))(x)
  x = layers.Flatten()(x)
  # x = layers.Dropout(rate=0.5)(x)
  x = layers.Dense(1000, activation='relu')(x)
  x = layers.Dense(10, activation='softmax')(x)

  inputs = dataset_cifar10
  model = models.Model(inputs, x, name='cnn_example_bn')
  return model

In [21]:
# create model instance
my_model = cnn_mobile_net(10)
my_model.summary()

Model: "cnn_mobile_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_s2 (Conv2D)            (None, 112, 112, 32)      896       
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
activation_112 (Activation)  (None, 112, 112, 32)      0         
_________________________________________________________________
conv2_dw_s1 (SeparableConv2D (None, 112, 112, 32)      1344      
_________________________________________________________________
bn_conv2 (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
activation_113 (Activation)  (None, 112, 112, 32)   

In [0]:
# run training
my_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

my_model.fit(dataset_cifar10, epochs=40, validation_data=(test_cifar10))

Epoch 1/40
    580/Unknown - 467s 805ms/step - loss: 2.0308 - accuracy: 0.2357